# __Chapter 13: Market-Making and Delta Hedging__

<br>

Finance 6470: Derivatives Markets <br>
Tyler J. Brough <br>
Last Update: April 18, 2019 <br>
<br>

## Python Imports

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.rcParams['figure.figsize'] = [20, 10]

In [29]:
def bsmCall(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    d2 = d1 - v * np.sqrt(t)
    prc = S * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
    
    return prc

In [30]:
def bsmDelta(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    d2 = d1 - v * np.sqrt(t)
    delta = np.exp(-q * t) * norm.cdf(d1) 
    
    return delta

## The Business of Option Market-Making

* Market makers provide liquidity to those who wish to buy and sell options. 

* They make their money through the bid-ask spread. 

* The position of the market maker is a result of whatever order flow arrives from customers.

* They can _"manufacture"_ options by synthetic trading.

* Their trading strategy is a relative-value strategy (betwen the bid and ask prices).

* They do not desire to speculate directionally on price movements. 

* An unhedged position therefore presents them with uncontrolled risk and may destroy their profits from the bid-ask spread.

* They typically engage in ___dynamic hedging___ to control for risks.

* __Q:__ What would Holbrook Working say about this kind of hedging strategy? 

* __Q:__ Does this remind you of any other dynamic trading strategies that we have examined? 

### Delta Hedging 

* Without hedging a market maker will have a completely arbitrary position driven by fulfilling customer orders. 

* An arbitrary portfolio has uncontrolled risks.

* Market makers engage in dynamic delta hedging to control risks.

* Delta hedging is also ___the key concept___ in equilibrium derivatives pricing models (e.g. Black-Scholes-Merton).

* McDonald pp. 383: ___"Delta-hedging is thus both a technique widely used in practice and a key to understanding option pricing."___

<br>

* Let's investigate delta hedging with a numerical example.

* For this example, assume the following:

<br>

<table style="width:15%">
    <caption style="text-align: left;"><b>Delta-Hedging Numerical Example</b></caption>
    <tbody>
    <tr>
        <th style="text-align: left;"><b>Item</b></th>
        <th style="text-align: left;"><b>Value</b></th>       
    </tr>
    <tr>
        <td style="text-align: left;">Spot Price</td>
        <td style="text-align: left;">\$40.00</td>       
    </tr>
    <tr>
        <td style="text-align: left;">Strike Price</td>       
        <td style="text-align: left;">\$40.0</td>
    </tr>
    <tr>
        <td style="text-align: left;">Volatility</td>                      
        <td style="text-align: left;">0.30</td>
    </tr>
    <tr>
        <td style="text-align: left;">Risk-Free Rate</td>                       
        <td style="text-align: left;">0.08</td>
    </tr>      
    <tr>
        <td style="text-align: left;">Dividend Yield</td>                    
        <td style="text-align: left;">0.0</td>
    </tr>    
    <tr>
        <td style="text-align: left;">Expiry</td>                          
        <td style="text-align: left;">91/365</td>
    </tr>                                
    </tbody>
</table>  

<br>

In [31]:
## Initiate variables as per the above table
S = 40.0
K = 40.0
v = 0.30
r = 0.08
q = 0.0
t = 91/365

The price, delta, gamma, and theta are given as follows:

<br>

<table style="width:15%">
    <caption style="text-align: left;"><b>Price and Greeks for Call Option</b></caption>
    <tbody>
    <tr>
        <th></th>
        <th style="text-align: left;"><b>Purchased</b></th>
        <th style="text-align: left;"><b>Written</b></th>       
    </tr>
    <tr>
        <th style="text-align: left;">Call Price</th>
        <td style="text-align: left;">2.7804</td>
        <td style="text-align: left;">-2.7804</td>       
    </tr>
    <tr>
        <th style="text-align: left;">Delta</th>
        <td style="text-align: left;">0.5824</td>       
        <td style="text-align: left;">-0.5824</td>
    </tr>    
    <tr>
        <th style="text-align: left;">Gamma</th>
        <td style="text-align: left;">0.0652</td>       
        <td style="text-align: left;">-0.0652</td>
    </tr>    
    <tr>
        <th style="text-align: left;">Theta</th>
        <td style="text-align: left;">-0.0173</td>       
        <td style="text-align: left;">0.0173</td>
    </tr>        
    </tbody>
</table>  

<br>

In [32]:
## Use BSM call formula to check above table
callPrc = bsmCall(S, K, v, r, t, q)
print(f"The Value of the European Call Option is: {callPrc : 0.04f}")

The Value of the European Call Option is:  2.7804


In [33]:
## Use BSM delta formula to check above table
callDelta = bsmDelta(S, K, v, r, t, q)
print(f"The Delta of the European Call Option is: {callDelta: 0.04f}")

The Delta of the European Call Option is:  0.5824


<br>

* We will assume that a customer arrives who wishes to purchase a 91-day 40-strike call option.

* The market-maker will fulfill the order by writing the option. 

* The Greek measures are the position are opposite of those for a purchased option.

* In particular, the delta of the market-maker's option delta is -0.5824.

* This means that the risk (for the market-maker) is that the stock price will rise. 

<br>

### An Example of Delta-Hedging for Two Days

* With the sale of a call, the market-maker is short delta shares (implicitly).

* To hedge this position, the market-maker will purchase (long) delta shares of the underlying stock to delta-hedge the position.

* Let's examine this hedging strategy as the market-maker marks-to-market daily.

<br>

#### __Day 0: Trade Initiation__


* Consider the 40-strike call above written on 100 shares of stock.

* When the market-maker sells the option she receives $287.04.

* Since $\Delta = 0.5824$, the market-maker buys 58.24 shares of the underlying stock (we will assume fractional shares can by purchased)

* The net investment is


$$
(58.24 \times \$40) - \$278.04 = \$2051.56
$$

* At an $8\%$ interest rate, the market-maker has an overnight financing charge of 

$$
\$2051.56 \times (e^{0.08/365} - 1) = \$0.45
$$



#### __Day 1: Marking-to-Market__

* We can first ask whether the market-maker made money or lost money overnight (we will worry about rebalancing in a minute).

* Suppose the new stock price at the close of market on day 1 is $40.50.

* The new call price with 1 day less to expiry is $3.0621.

* Overnight PnL is a gain of $0.50, computed as follows:

<table style="width:20%">
    <tbody>
    <tr>
        <td style="text-align: left;">Gain on 58.24 shares</td>
        <td style="text-align: right;">58.24 x (\$40.50 - \$40.00) = \$29.12</td>       
    </tr>
    <tr>
        <td style="text-align: left;">Gain on written call option</td>       
        <td style="text-align: right;">\$278.04 - \$306.21 = -\$28.17</td>
    </tr>    
    <tr>
        <td style="text-align: left;">Interest</td>       
        <td style="text-align: right;">\$2051.56 $\times (e^{0.08/365} - 1)$ = -\$0.45</td>
    </tr>    
    <tr>
        <td style="text-align: left;"><b>Overnight profit</b></td>       
        <td style="text-align: right;"><b>\$0.50</b></td>
    </tr>        
    </tbody>
</table> 

<br>


#### __Day 1: Rebalancing the Portfolio__

* The new delta is 0.6142 (at the new price of $\$40.50$ and with 1 day less to expiry)

* The market-maker therefore must purchase an additional $(61.42 - 58.24) = 3.18$ shares of the underlying stock

* This rebalancing requires an additional injection of capital, equal to $\$40.50 \times 3.18 = \$128.79$.

* Since the readjustment in the number of shares entails buying at the current price, it does not affect market-to-market profits for that day.

<br>

Let's see these calculations:

In [42]:
## Get Call Price and Delta for New Spot and Expiry
S = 40.50
t = 90/365
callPrc = bsmCall(S, K, v, r, t, q)
print(f"The Value of the European Call Option is: {callPrc : .04f}")

callDelta = bsmDelta(S, K, v, r, t, q)
print(f"The Delta of the European Call Option is: {callDelta: .04f}")

The Value of the European Call Option is:  3.0621
The Delta of the European Call Option is:  0.6142


<br>

#### __Day 2__

* Assume that the stock price now falls to $\$39.25$

* The market-maker makes money on the written option, but loses money on the 61.42 shares.

* Interest expense has increased due to the additional capital required to purchase additional shares from rebalancing the day before. 

* The net result from marking-to-market is a loss of $-\$3.87$:


<table style="width:20%">
    <tbody>
    <tr>
        <td style="text-align: left;">Gain on 61.42 shares</td>
        <td style="text-align: right;">58.24 x (\$39.25 - \$40.50) = \$76.78</td>       
    </tr>
    <tr>
        <td style="text-align: left;">Gain on written call option</td>       
        <td style="text-align: right;">\$306.21 - \$232.82 = \$73.39</td>
    </tr>    
    <tr>
        <td style="text-align: left;">Interest</td>       
        <td style="text-align: right;">\$2181.30 $\times (e^{0.08/365} - 1)$ = -\$0.48</td>
    </tr>    
    <tr>
        <td style="text-align: left;"><b>Overnight profit</b></td>       
        <td style="text-align: right;"><b>-\$3.87</b></td>
    </tr>        
    </tbody>
</table>

<br>

In [35]:
## Get Call Price and Delta for New Spot and Expiry
S = 39.25
t = 89/365
callPrc = bsmCall(S, K, v, r, t, q)
print(f"The Value of the European Call Option is: {callPrc : .04f}")

callDelta = bsmDelta(S, K, v, r, t, q)
print(f"The Delta of the European Call Option is: {callDelta: .04f}")

The Value of the European Call Option is:  2.3282
The Delta of the European Call Option is:  0.5311


<br>

* This process rolls on in like manner until the expiration date is reached.

<br>


### Interpreting the Profit Calculation

As time passes, there are three main sources of cash flow into and out of the portfolio:

1. __Borrowing:__ The market-maker's borrowing capacity equals the value of the securities held in the portfolio. Thus, this value changes each day with the new market data. On day 0, the net value of the portfolio was $\$2051.56$. At the close of trading on day 1, the share price increased to $\$40.50$ and we had to purchase $3.18$ shares to rebalance the portfolio for a total of $61.42$ shares. The market value of the portfolio was $(61.42 \times \$40.50) - \$306.21 = \$2181.30$. Her borrowing capacity increased by $\$129.74$. 

2. __Purchase or sale of shares:__ The market-maker will buy or sell shares each day (or some other duration) as necessary to maintain delta-neutrality. In the above example, she had to purchase an additional $3.18$ shares on day 1. The price at the time was $\$40.50$, so the cost was $3.18 \times \$40.50 = \$128.79$. 

3. __Interest:__ She pays interest on the borrowed amount. On day 1 she owed $\$0.45$.

<br>

Thus, she needs $\$128.79$ to buy more shares and $\$0.45$ to pay the interest cost. The change in her borrowing capacity is $\$129.74$. The difference between what the bank (or broker) will lend her on the one hand and the cost of additional shares plus interest on the other is:

$$
\$129.74 - \$128.79 - \$0.45 = \$0.50
$$

Thus, because she can borrow against the value of the portfolio she is free to pocket this difference as profit. 

<br>

* Mark-to-market profit equals the net cash flow generated by borrowing to fully fund the position.

* We can also get at this by looking at the source and use of funds, and the extent to which it is necessary to inject additional cash to maintain the delta-hedge.

* We can calculate net cash flow from the portfolio as:

$$
\begin{align}
\mbox{Net cash flow} &= \mbox{Change in borrowing capacity} \\
                     & - \mbox{cash used to purchase additional shares} \\
                     & - \mbox{interest}
\end{align} 
$$

<br>

Let:

* $\Delta_{i}$ denote the option delta on day $i$

* $S_{i}$ denote the stock price

* $C_{i}$ denote the option price

* $MV_{i}$ denote the market value of the portfolio

Borrowing capacity on day $i$ is:

$$
MV_{i} = \Delta_{i} S_{i} - C_{i}
$$

Hence, the change in borrowing capacity from day to day is:

$$
MV_{i} - MV_{i-1} = \Delta_{i} S_{i} - C_{i} - (\Delta_{i-1} S_{i-1} - C_{i-1})
$$

* The cost purchasing additional shares is: $S_{i} (\Delta_{i} - \Delta_{i-1})$

* Interest owed on day $i$, which depends on the previous day's borrowing, is: $rMV_{i-1}$

$$
\begin{align}
\mbox{Net cash flow} &= MV_{i} - MV_{i-1} - S_{i}(\Delta_{i} - \Delta_{i-1}) - rMV_{i-1} \\
                     &= \Delta_{i} S_{i} - C_{i} - (\Delta_{i-1} S_{i-1} - C_{i-1}) - S_{i}(\Delta_{i} - \Delta_{i-1}) - rMV_{i-1} \\
                     &= \Delta_{i-1} (S_{i} - S_{i-1}) - (C_{i} - C_{i-1}) - rMV_{i-1}
\end{align}
$$

This last expression is equal to the:

* overnight gain on shares

* less the overnight gain on the option

* less interest

In the numerical example above, we have the values:

$$
\begin{align}
MV_{1} - MV_{0} - S_{1}(\Delta_{1} - \Delta_{0}) - rMV_{0} &= \$2181.30 - \$2051.56 - \$128.79 - \$0.45 \\
                                                           &= \$0.50
\end{align}                                                          
$$

Which, as can be seen, is equal to the overnight PnL we calculated above between day 0 and day 1.

In [39]:
## Check rMV calcualation
print(f"Overnight interest at 8% on $2051.56: {(0.08 / 365) * 2051.56 : 0.2f}")

Overnight interest at 8% on $2051.56:  0.45


<br>

## __Delta Hedging for Several Days__

* Let's examine this process for 5 consecutive days

* Hypothetical closing prices given on row 1

* The last row gives the daily PnL

<br>
<br>

<table style="width:40%">
    <caption style="text-align: left;"><b>The Market Maker's Daily Profit Over 5 Days of Delta Hedging</b></caption>
    <tbody>
    <tr>
        <th></th>
        <th><b>Day 0</b></th>
        <th><b>Day 1</b></th>
        <th><b>Day 2</b></th>
        <th><b>Day 3</b></th>        
        <th><b>Day 4</b></th>        
        <th><b>Day 5</b></th>        
    </tr>
    <tr>
        <th style="text-align: left;">Stock (\$)</th>
        <td style="text-align: center;">40.00</td>
        <td style="text-align: center;">40.50</td>        
        <td style="text-align: center;">39.25</td>        
        <td style="text-align: center;">38.75</td>        
        <td style="text-align: center;">40.00</td>
        <td style="text-align: center;">40.00</td>        
    </tr>
    <tr>
        <th style="text-align: left;">Call (\$)</th>
        <td style="text-align: center;">278.04</td>
        <td style="text-align: center;">306.21</td>
        <td style="text-align: center;">232.82</td>
        <td style="text-align: center;">205.46</td>
        <td style="text-align: center;">271.04</td>        
        <td style="text-align: center;">269.27</td>        
    </tr>
    <tr>
        <th style="text-align: left;">100 x Delta</th>
        <td style="text-align: center;">58.24</td>
        <td style="text-align: center;">61.42</td>
        <td style="text-align: center;">53.11</td>
        <td style="text-align: center;">49.56</td>
        <td style="text-align: center;">58.06</td>
        <td style="text-align: center;">58.01</td>                        
    </tr>
    <tr>
        <th style="text-align: left;">Investment (\$)</th>
        <td style="text-align: center;">2051.58</td>
        <td style="text-align: center;">2181.30</td>
        <td style="text-align: center;">1851.65</td>
        <td style="text-align: center;">1715.12</td>
        <td style="text-align: center;">2051.35</td>
        <td style="text-align: center;">2051.29</td>                        
    </tr>      
    <tr>
        <th style="text-align: left;">Interest (\$)</th>
        <td style="text-align: center;"></td> 
        <td style="text-align: center;">-0.45</td>
        <td style="text-align: center;">-0.48</td>
        <td style="text-align: center;">-0.41</td>
        <td style="text-align: center;">-0.38</td>
        <td style="text-align: center;">-0.45</td>                   
    </tr>    
    <tr>
        <th style="text-align: left;">Capital Gain (\$)</th>
        <td style="text-align: center;"></td>
        <td style="text-align: center;">0.95</td> 
        <td style="text-align: center;">-3.39</td>
        <td style="text-align: center;">0.81</td>
        <td style="text-align: center;">-3.62</td>
        <td style="text-align: center;">1.77</td>                           
    </tr>         
     <tr>
        <th style="text-align: left;">Daily Proft (\$)</th>
        <td style="text-align: center;"></td>
        <td style="text-align: center;"><b>\$0.50</b></td> 
        <td style="text-align: center;"><b>-\$3.87</b></td>
        <td style="text-align: center;"><b>\$0.40</b></td>
        <td style="text-align: center;"><b>-\$4.00</b></td>
        <td style="text-align: center;"><b>\$1.32</b></td>                           
    </tr>                        
    </tbody>
</table>  


<br>
<br>

<br>

What determines the pattern of PnL in the table? There are tree main factors: gamma, theta, and the carrying cost of the hedging position.


<br>


1. __Gamma:__ For the largest moves in the stock, the market-maker loses money. For small moves, she makes money. The loss from the large moves is due to gamma (the market-maker is said to be __short gamma__). From McDonald pp. 388, ___"As the stock price rises, the delta of the call increases and the call loses money faster than the stock makes money. As the stock price falls, the delta of the call decreases and the call makes money more slowly than the fixed stock position loses money."___ She becomes unhedged net long as the stock price falls and unhedged net short as the stock price rises. The losses on days 2 and 4 are due to gamma. 


2. __Theta:__ When a day passes with no other changes, the option becomes cheaper. Time decay works to the benefit of the market-maker (in this example where she is short the option). That is because she can close out the position with the purchase of an equivalent option that is now cheaper. This is especially evident on day 5, but also on days 1 and 3. 


3. __Interest Cost:__ The market-maker must borrow to buy shares of stock to maintain the delta-hedge. The net carrying cost is a component of the overall cost.

<br>